In [1]:
import datetime
from dateutil.relativedelta import *
from fuzzywuzzy import fuzz
import glob
import numpy as np
import pandas as pd
from scipy.stats import ttest_1samp
import sys
import xarray as xr


from paths_bra import *

sys.path.append('./..')
from refuelplot import *
setup()

from utils import *

In [2]:
gen_path = bra_path + '/generation'

In [3]:
## load generation data
# load usinas hourly
if gen_path + '/hourly/usinas.pkl' not in glob.glob(gen_path + '/hourly/*.pkl'):
    USIh = pd.read_csv(gen_path + '/hourly/Comparativo_Geração_de_Energia_Semana_data_usinas.csv',
                       sep = ';', index_col = 0, parse_dates = True, dayfirst = True).iloc[1:,[6,8]].sort_index()
    # remove missing values
    USIh = USIh.loc[USIh.index.notnull()].dropna()
    USIh.columns = ['usina','prod_GWh']

    # in RIO DO FOGO there is one duplicate hour after one missing hour -> change timestamps of those hours
    idxUSIh = USIh.index.values
    midxUSIh = USIh.reset_index().set_index(['usina','Data Escala de Tempo 1 GE Comp 3']).index
    idxUSIh[midxUSIh.duplicated(keep='last')]  = idxUSIh[midxUSIh.duplicated(keep='first')] - np.timedelta64(1,'h')
    USIh.index = pd.DatetimeIndex(idxUSIh)

    USIhs = USIh.reset_index().set_index(['usina','index']).unstack(level=0).prod_GWh
    USIhs.to_csv(gen_path + '/hourly/usinas.csv')
    USIhs.to_pickle(gen_path + '/hourly/usinas.pkl')
#USIhs = pd.read_csv(gen_path + '/hourly/usinas.csv',index_col=0,parse_dates=True)
USIhs = pd.read_pickle(gen_path + '/hourly/usinas.pkl')

In [4]:
# load usinas monthly
if gen_path + '/monthly/usinas.pkl' not in glob.glob(gen_path + '/monthly/*.pkl'):
    USIm = pd.read_csv(gen_path + '/monthly/Comparativo_Geração_de_Energia_Semana_data_usinas.csv',
                         sep = ';', index_col = 0, parse_dates = True, dayfirst = True).iloc[1:,[6,8]].sort_index()
    # remove missing values
    USIm = USIm.loc[USIm.index.notnull()].dropna()
    USIm.columns = ['usina','prod_GWh']

    USIms = USIm.reset_index().set_index(['usina','Data Escala de Tempo 1 GE Comp 3']).unstack(level=0).prod_GWh
    USIms.to_csv(gen_path + '/monthly/usinas.csv')
    USIms.to_pickle(gen_path + '/monthly/usinas.pkl')
#USIms = pd.read_csv(gen_path + '/monthly/usinas.csv',index_col=0,parse_dates=True)
USIms = pd.read_pickle(gen_path + '/monthly/usinas.pkl')

In [5]:
USIhs[USIhs.fillna(0).cumsum(axis=0)==0] = np.nan # remove leading 0s
USIhs[USIhs[::-1].fillna(0).cumsum(axis=0)[::-1]==0] = np.nan # remove trailing 0s

USIms[USIms.fillna(0).cumsum(axis=0)==0] = np.nan # remove leading 0s
USIms[USIms[::-1].fillna(0).cumsum(axis=0)[::-1]==0] = np.nan # remove trailing 0s

In [6]:
def get_cap_df(cap,comdate):
    com = pd.DataFrame({'capacity': cap}).groupby(comdate).sum()
    cap_cum = com.capacity.cumsum()
    # if only years given for commissioning dates -> gradual capacity increase over year, full capacity at end of year
    if type(cap_cum.index.values[0]) == np.int64:
        cap_cum.index = [np.datetime64(str(int(year))+"-12-31 23:00:00") for year in cap_cum.index.values]
        # create yearly dates at yearends
        drcc = pd.date_range(np.datetime64('2005-12-31 23:00:00'),
                             np.datetime64('2019-12-31 23:00:00'),freq= 'y')
        cap_cum = pd.Series(drcc.map(cap_cum),index = drcc)
        # if first year emtpy: either year before or 0 if nothing before
        if(sum(com.index<2000) > 0):
            cap_cum[0] = com.cumsum()[com.index<2000].max()
        else:
            cap_cum[0] = 0
        # if missing years -> put capacity of year before
        cap_cum = cap_cum.ffill()
    dr = pd.date_range('1/1/2006','31/12/2019 23:00:00',freq = 'h')
    cap_ts = pd.Series(dr.map(cap_cum),index = dr)
    cap_ts[0] = cap_cum[cap_cum.index<=pd.Timestamp('2006-01-01')].max()
    if type(comdate[0]) == np.int64:
        return(cap_ts.interpolate(method='linear'))
    else:
        return(cap_ts.fillna(method='ffill'))

In [7]:
# load and match aneel and ons windparks
def matchWords(word, statements):
    # function to match a word to different statements
    # output: ratio of matching (0-100) for all provided statements
    results = []
    for s in statements:
        r = fuzz.ratio(word, s)
        results.append(r)
    return results

def match_string(string, array):
    # function for matching casefolded strings
    Slc = string.strip().casefold()
    Alc = [arr.casefold() for arr in array.str.strip().unique()]
    scores = matchWords(Slc, Alc)
    mscore = max(scores)
    strarr = array.unique()[np.where(np.array(scores)==mscore)][0]
    return(string,strarr,mscore)

def match_anl(string):
    # function to match ONS to ANL windparks
    return(match_string(string,ANL2.name))

# load ANEEL and ONS windparks
ONS = pd.read_csv(bra_path + '/ONS_windparks.csv', index_col = 0)
# remove those with CONJUNTO EOLICO - they're there twice and capacities don't match with ANEEL data
ONS = ONS[~ONS.usina.str.contains('CONJUNTO EOLICO')]
# remove some other duplicate windparks
ONS = ONS[[d not in [' CANOA QUEBRADA (E-RV-ACEP)',' PV DO NORDESTE',' SM (SANTA MARIA)',' SÃO BENTO NORTE II'] for d in ONS.usina]]
ANL = pd.read_csv(bra_path + '/turbine_data.csv', index_col = 0)

# characters and strings to replace for better matching
letters = {'Ãµ':'õ',
           'ó':'o',
           'ã':'a',
           'á':'a',
           'â':'a',
           'é':'e',
           'Ã':'A',
           'Á':'A',
           'Â':'A',
           'Ó':'O',
           'É':'E',
           'ú':'u',
           'ô':'o',
           'Ô':'O',
           'ú':'u',
           'Ú':'U',
           'ç':'c',
           'Ç':'C',
           'í':'i',
           'Í':'I',
           'Ê':'E'}
remove = {' 2LER':'',
          ' 2LFA':'',
          ' LFA':'',
          'EOL ':'',
          ' 3LER':'',
          'Usina Eolica ':'',
          'Eólica ':'',
          ' ENERGIAS RENOVAVEIS':'',
#          ' CONJUNTO EOLICO':'',
          '\(E-BV-ACEP\)':'',
          '\(E-RV-ACEP\)':'',
          '\(BELA BISTA\)':'',
          '\(ENERGEN\)':'',
          '\(Antiga Ventos Maranhenses 05\)':'',
          'PARQUE EOLICO ':'',
          ' - N HORIZ':'',
          'ENERGETICA S/A':'',
          '\(ILHEUS\)':'',
          ' EOLOS':'',
          'S\.A\.':''}
replace = {'LAG DO':'LAGOA DO',
           'VENTOS S VICENTE':'VENTOS DE SAO VICENTE',
           'SERRA BABILONIA':'SERRA DA BABILONIA',
           'CORREDOR SENANDES':'CORREDOR DO SENANDES',
           'SAO BENTO NORTE':'SAO BENTO DO NORTE',
           'GAMELEIRAS':'GAMELERIAS',
           'Lagoinha':'Lagoinh',
           'PAPAGAIOS':'PAPAGAIO',
           'VENTOS DE SAO ABRAAO':'VENTOS DO SANTO ABRAAO',
           'VENTOS DO SAO MARIO':'VENTOS DE SAO MARIO',
           'DAGUA':'D AGUA',
           'B VEN':'BONS VENTOS',
           'NOVA BURITI':'BURITI',
           'NOVA CAJUCOCO':'CAJUCOCO',
           'PALMAS':'DE PALMAS',
           'DE PALMARES':'PALMARES',
           'PV DO NORDESTE':'VENTOS DO NORDESTE',
           'Aura Lagoa do Barro':'Lagoa do Barro',
           'AURA LAGOA DO BARRO':'LAGOA DO BARRO',
           'LAGOA BARRO':'LAGOA DO BARRO',
           'GRAVATA':'GRAVATA FRUITRADE',
           'FAZENDA DO ROSARIO':'FAZENDA ROSARIO',
           'Parque Eolico do Horizonte':'Ventos de Horizonte',
           'S BENTO':'SAO BENTO',
           'SANTO ANTONIO (BTG PACTUAL)':'SANTO ANTONIO DE PADUA',
           'SM \(SANTA MARIA\)':'SANTA MARIA',
           'SAO JORGE CE':'SAO JORGE',
           'VENT DA ST ESPERANCA':'VENTOS DA SANTA ESPERANCA',
           'VENTOS DA STA DULCE':'VENTOS DA SANTA DULCE',
           'ESPERANCA NORDESTE':'ESPERANCA DO NORDESTE',
           'Eolica Delta':'Delta',
           'Eolica Serra das Vacas':'Serra das Vacas',
           'Ventos de Santo Augusto':'Santo Augusto',
           'Ventos do Sao Gabriel':'Sao Gabriel',
           'GE Maria Helena':'Maria Helena'}
numbers = {'10':'X',
           '11':'XI',
           '12':'XII',
           '13':'XIII',
           '14':'XIV',
           '15':'XV',
           '17':'XVII',
           '19':'XIX',
           '21':'XXI',
           '23':'XXIII',
           '24':'XXIV',
           '25':'XXV',
           '26':'XXVI',
           '27':'XXVII',
           '28':'XXVIII',
           '29':'XXIX',
           '31':'XXXI',
           '34':'XXXIV',
           '35':'XXXV',
           '36':'XXXVI',
           '01':'I',
           '02':'II',
           '03':'III',
           '04':'IV',
           '05':'V',
           '06':'VI',
           '07':'VII',
           '08':'VIII',
           '09':'IX',
           '1':'I',
           '2':'II',
           '3':'III',
           '4':'IV',
           '5':'V',
           '6':'VI',
           '7':'VII',
           '8':'VIII',
           '9':'IX'}

# replace characters
ONS2 = ONS.copy(deep=True)
ANL2 = ANL.copy(deep=True)
for i in letters:
    ONS2.usina = ONS2.usina.str.replace(i,letters.get(i))
    ANL2.name = ANL2.name.str.replace(i,letters.get(i))
for i in replace:
    ONS2.usina = ONS2.usina.str.replace(i,replace.get(i))
    ANL2.name = ANL2.name.str.replace(i,replace.get(i))
for i in remove:
    ONS2.usina = ONS2.usina.str.replace(i,remove.get(i))
for i in numbers:
    ONS2.usina = ONS2.usina.str.replace(i,numbers.get(i))
    ANL2.name = ANL2.name.str.replace(i,numbers.get(i))

# match windparks
matches = ONS2.usina.apply(match_anl).apply(pd.Series)
matches.columns = ['ONS_name','ANL_name','score']
len(matches[matches.score<100])

53

In [8]:
ONSd = pd.Series(ONS.usina.values,index=ONS2.usina.values)#.reset_index().drop_duplicates()
ANLd = pd.Series(ANL.name.values,index=ANL2.name.values)#.reset_index().drop_duplicates()
ONSd.columns = ['simpl','orig']
ANLd.columns = ['simpl','orig']

In [9]:
matches2 = pd.DataFrame({'ANL_name':matches.ANL_name.map(ANLd.drop_duplicates()),
                         'ONS_name':matches.ONS_name.map(ONSd),
                         'score':matches.score})

### get only matching power generation timeseries

In [10]:
wpUSIm = USIms[matches2.ONS_name[matches2.score==100].values]

In [11]:
matches2H = matches2[[usi in USIhs.columns.values for usi in matches2.ONS_name]]

In [12]:
wpUSIh = USIhs[matches2H.ONS_name.values]

### load simualted data

In [13]:
wpERAxr = xr.open_dataset(results_path + '/windpower_stat_ERA5.nc')
wpMERxr = xr.open_dataset(results_path + '/windpower_stat_MERRA2.nc')
wpERAgxr = xr.open_mfdataset(results_path +'/windpower_??_ERA5_GWA.nc')
wpMERgxr = xr.open_mfdataset(results_path +'/windpower_??_MERRA2_GWA.nc')

In [14]:
turb_mer = pd.read_csv(bra_path + '/turbine_data_mer.csv',index_col=0)
turb_era = pd.read_csv(bra_path + '/turbine_data_era.csv',index_col=0)
turb_merg = pd.read_csv(bra_path + '/turbine_data_mer_gwa3.csv',index_col=0)
turb_erag = pd.read_csv(bra_path + '/turbine_data_era_gwa3.csv',index_col=0)

In [15]:
lbl = pd.read_csv(bra_path+ '/labels_turbine_data_gwa3.csv',index_col=0)

In [16]:
# prepare simulated data as dataframe
wpMERdf = wpMERxr.to_dataframe().unstack().wp
wpERAdf = wpERAxr.to_dataframe().unstack().wp
wpMERgdf = wpMERgxr.assign_coords(location=range(len(wpMERgxr.location.values))).to_dataframe().unstack().wp
wpERAgdf = wpERAgxr.assign_coords(location=range(len(wpERAgxr.location.values))).to_dataframe().unstack().wp
# some locations have more than one park, get shares of parks
sharesMER = ANL.cap.groupby([lbl.lbl_mer.values,ANL.name.values]).sum()/ANL.cap.groupby([lbl.lbl_mer.values,ANL.name.values]).sum().index.get_level_values(0).map(ANL.cap.groupby(lbl.lbl_mer.values).sum())
sharesERA = ANL.cap.groupby([lbl.lbl_era.values,ANL.name.values]).sum()/ANL.cap.groupby([lbl.lbl_era.values,ANL.name.values]).sum().index.get_level_values(0).map(ANL.cap.groupby(lbl.lbl_era.values).sum())
sharesMERg = ANL.cap.groupby([lbl.lbl_mer_gwa.values,ANL.name.values]).sum()/ANL.cap.groupby([lbl.lbl_mer_gwa.values,ANL.name.values]).sum().index.get_level_values(0).map(ANL.cap.groupby(lbl.lbl_mer_gwa.values).sum())
sharesERAg = ANL.cap.groupby([lbl.lbl_era_gwa.values,ANL.name.values]).sum()/ANL.cap.groupby([lbl.lbl_era_gwa.values,ANL.name.values]).sum().index.get_level_values(0).map(ANL.cap.groupby(lbl.lbl_era_gwa.values).sum())
# get generation per park
wpMER = wpMERdf.loc[sharesMER.index.codes[0].values()].mul(sharesMER.values,axis=0).groupby(sharesMER.index.get_level_values(1).values).sum().transpose()
wpERA = wpERAdf.loc[sharesERA.index.codes[0].values()].mul(sharesERA.values,axis=0).groupby(sharesERA.index.get_level_values(1).values).sum().transpose()
wpMERg = wpMERgdf.loc[sharesMERg.index.codes[0].values()].mul(sharesMERg.values,axis=0).groupby(sharesMERg.index.get_level_values(1).values).sum().transpose()
wpERAg = wpERAgdf.loc[sharesERAg.index.codes[0].values()].mul(sharesERAg.values,axis=0).groupby(sharesERAg.index.get_level_values(1).values).sum().transpose()
# adapt index of MERRA data in 2019 (substract half an hour)
wpMER.index = wpMER.index[wpMER.index<'2018-12'].append(wpMER.index[wpMER.index>='2018-12'] - np.timedelta64(30,'m'))
wpMERg.index = wpMER.index[wpMERg.index<'2018-12'].append(wpMERg.index[wpMERg.index>='2018-12'] - np.timedelta64(30,'m'))
# set time zones
wpMER = wpMER.tz_localize('UTC').tz_convert('Etc/GMT-3')
wpERA = wpERA.tz_localize('UTC').tz_convert('Etc/GMT-3')
wpMERg = wpMERg.tz_localize('UTC').tz_convert('Etc/GMT-3')
wpERAg = wpERAg.tz_localize('UTC').tz_convert('Etc/GMT-3')
wpUSIh = wpUSIh.tz_localize('Etc/GMT-3')
wpUSIm = wpUSIm.tz_localize('Etc/GMT-3')

In [83]:
def analyseUSIh(parks):
    compUSIh= pd.DataFrame({'MERRA2':wpMER[parks.ANL_name],
                            'ERA5':wpERA[parks.ANL_name],
                            'MERRA2_GWA':wpMERg[parks.ANL_name],
                            'ERA5_GWA':wpERAg[parks.ANL_name],
                            'wp_obs':wpUSIh[parks.ONS_name]*10**6})
    # get capacities
    capUSIh = get_cap_df(ANL[ANL.name==parks.ANL_name].cap.values,
                         ANL[ANL.name==parks.ANL_name].commissioning.astype(np.datetime64).values).tz_localize('UTC').tz_convert('Etc/GMT-3')
    # calculate capacity factors
    cf_USIh = compUSIh.div(capUSIh,axis=0).dropna()
    stat_h = pd.DataFrame({'ERA5':stats(cf_USIh.ERA5,cf_USIh.wp_obs,False),
                           'ERA5_GWA':stats(cf_USIh.ERA5_GWA,cf_USIh.wp_obs,False),
                           'MERRA2':stats(cf_USIh.MERRA2,cf_USIh.wp_obs,False),
                           'MERRA2_GWA':stats(cf_USIh.MERRA2_GWA,cf_USIh.wp_obs,False),
                           'obs':[np.nan,np.nan,np.nan,cf_USIh.wp_obs.mean()]},
                          index = ['cor','rmse','mbe','avg']).reset_index().melt(id_vars=['index']).dropna()
    stat_h.columns = ['param','dataset',parks.ANL_name]
    return(stat_h.set_index(['param','dataset']).transpose())

In [119]:
def analyseUSId(parks):
    compUSIh= pd.DataFrame({'MERRA2':wpMER[parks.ANL_name],
                            'ERA5':wpERA[parks.ANL_name],
                            'MERRA2_GWA':wpMERg[parks.ANL_name],
                            'ERA5_GWA':wpERAg[parks.ANL_name],
                            'wp_obs':wpUSIh[parks.ONS_name]*10**6})
    # get capacities
    capUSIh = get_cap_df(ANL[ANL.name==parks.ANL_name].cap.values,
                         ANL[ANL.name==parks.ANL_name].commissioning.astype(np.datetime64).values).tz_localize('UTC').tz_convert('Etc/GMT-3')
    ccUSIh = pd.concat([compUSIh,capUSIh],axis=1).dropna()
    ccUSIh.columns = compUSIh.columns.tolist() + ['cap']
    # aggregate daily
    ccUSId = ccUSIh.resample('D').sum()
    cf_USId = ccUSId.drop('cap',axis=1).div(ccUSId.cap,axis=0)
    stat_d = pd.DataFrame({'ERA5':stats(cf_USId.ERA5,cf_USId.wp_obs,False),
                           'ERA5_GWA':stats(cf_USId.ERA5_GWA,cf_USId.wp_obs,False),
                           'MERRA2':stats(cf_USId.MERRA2,cf_USId.wp_obs,False),
                           'MERRA2_GWA':stats(cf_USId.MERRA2_GWA,cf_USId.wp_obs,False),
                           'obs':[np.nan,np.nan,np.nan,cf_USId.wp_obs.mean()]},
                          index = ['cor','rmse','mbe','avg']).reset_index().melt(id_vars=['index']).dropna()
    stat_d.columns = ['param','dataset',parks.ANL_name]
    return(stat_d.set_index(['param','dataset']).transpose())

In [ ]:
stats_USIh = pd.concat(matches2H[matches2H.score==100].apply(analyseUSIh,axis=1).tolist(),axis=0).transpose()

In [121]:
stats_USId = pd.concat(matches2H[matches2H.score==100].apply(analyseUSId,axis=1).tolist(),axis=0).transpose()